In [1]:
import pandas as pd
import torch

# Part 1: Predicting with Hardcoded Model

In [2]:
# Loading train.csv to DataFrame and constructing a trainX tensor
train_df = pd.read_csv("train.csv")
train_tensor = torch.tensor(train_df.values, dtype=torch.float64)
trainX = train_tensor[:, [0,9]]
trainX

tensor([[ 24.,   0.],
        [ 22.,   0.],
        [ 84.,   8.],
        ...,
        [268.,   7.],
        [ 81.,   0.],
        [118.,   0.]], dtype=torch.float64)

In [3]:
# Constructing a trainY tensor
trainY = train_tensor[:, 10]
trainY.reshape(1044, -1)

tensor([[3.],
        [2.],
        [9.],
        ...,
        [5.],
        [2.],
        [5.]], dtype=torch.float64)

In [4]:
# Loading test.csv to DataFrame and constructing a testX tensor
test_df = pd.read_csv("test.csv")
test_tensor = torch.tensor(test_df.values, dtype=torch.float64)
testX = test_tensor[:, [0, 9]]
testX

tensor([[ 53.,   0.],
        [ 48.,  67.],
        [ 45.,   0.],
        [ 61.,   0.],
        [123.,   9.],
        [ 98.,   5.],
        [118.,  29.],
        [ 51.,   0.],
        [  0.,   0.],
        [214.,   0.],
        [163.,  22.],
        [199.,  12.],
        [260.,  29.],
        [167.,  42.],
        [129.,  12.],
        [127.,   8.],
        [109.,   0.],
        [ 90.,  14.],
        [ 46.,   0.],
        [150.,  14.],
        [110.,  10.],
        [  0.,  26.],
        [142.,  14.],
        [159.,   0.],
        [465.,  21.],
        [ 94.,   0.],
        [ 92.,  20.],
        [ 39.,   0.],
        [268.,   5.],
        [ 91.,   0.],
        [162.,   0.],
        [ 49.,   0.],
        [ 79.,   5.],
        [ 84.,   5.],
        [ 38.,   0.],
        [125.,   6.],
        [351.,   9.],
        [ 80.,  38.],
        [ 49.,   0.],
        [110.,   0.],
        [158.,   0.],
        [ 46.,   5.],
        [242.,  31.],
        [ 89.,   9.],
        [147.,  21.],
        [2

In [5]:
# Constructing a testY tensor
testY = test_tensor[:, 10]
testY.reshape(348, -1)

tensor([[ 9.],
        [26.],
        [ 2.],
        [ 7.],
        [ 9.],
        [13.],
        [17.],
        [ 4.],
        [ 0.],
        [ 9.],
        [13.],
        [ 9.],
        [27.],
        [24.],
        [14.],
        [11.],
        [ 3.],
        [23.],
        [ 3.],
        [16.],
        [13.],
        [27.],
        [12.],
        [10.],
        [17.],
        [ 7.],
        [22.],
        [ 2.],
        [ 4.],
        [ 4.],
        [ 5.],
        [ 4.],
        [ 5.],
        [ 4.],
        [ 1.],
        [13.],
        [ 5.],
        [18.],
        [ 6.],
        [ 4.],
        [ 8.],
        [ 6.],
        [14.],
        [ 9.],
        [13.],
        [19.],
        [17.],
        [22.],
        [27.],
        [10.],
        [18.],
        [ 7.],
        [ 5.],
        [16.],
        [ 4.],
        [12.],
        [15.],
        [10.],
        [17.],
        [13.],
        [ 8.],
        [ 2.],
        [ 6.],
        [13.],
        [ 9.],
        [12.],
        [ 

In [6]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=testX.dtype)
coef

tensor([[0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0300],
        [0.0300],
        [0.0300],
        [0.0300]], dtype=torch.float64)

In [7]:
if torch.cuda.is_available():
    testX = testX.to("cuda")
    coef = coef.to("cuda")
predicted_deaths = testX[0] * coef
predicted_deaths

tensor([[0.2120, 0.0000],
        [0.2120, 0.0000],
        [0.2120, 0.0000],
        [0.2120, 0.0000],
        [0.2120, 0.0000],
        [0.2120, 0.0000],
        [1.5900, 0.0000],
        [1.5900, 0.0000],
        [1.5900, 0.0000],
        [1.5900, 0.0000]], dtype=torch.float64)

# Part 2: R^2 Score

In [10]:
# Creating a predictedY tensor
coef = coef.reshape(2, -1)
predictedY = testX @ coef
predictedY

tensor([[0.2120, 0.2120, 0.2120, 0.2120, 0.2120],
        [0.4600, 2.2020, 2.2020, 2.2020, 2.2020],
        [0.1800, 0.1800, 0.1800, 0.1800, 0.1800],
        ...,
        [0.8480, 1.0560, 1.0560, 1.0560, 1.0560],
        [0.2360, 0.4700, 0.4700, 0.4700, 0.4700],
        [0.1200, 0.1200, 0.1200, 0.1200, 0.1200]], dtype=torch.float64)

In [11]:
# Function to calculate the R^2 score for any vector
def r2_score(testY, predictedY):
    SStot = 0
    SSreg = 0
    rsquared_score = 0
    
    if torch.cuda.is_available():
        testY = testY.to("cuda")
        
    agg = testY.sum()
    num_elements = torch.numel(testY)
    testY_average = agg / num_elements
    
    for val in testY:
        SStot_difference = val - testY_average
        SSreg_difference = testY - predictedY
        SStot = SStot + pow(SStot_difference, 2)
        SSreg = SSreg + pow(SSreg_difference, 2)
    
    improvement = SStot - SSreg
    rsquared_score = improvement / SStot

r2_score(testY, predictedY)

RuntimeError: The size of tensor a (348) must match the size of tensor b (5) at non-singleton dimension 1